In [2]:
import pandas as pd
import plotly.express as px
from plotly.io import show
import plotly.graph_objects as go
from fastf1.ergast import Ergast

SEASON = 2022

In [34]:
ergast = Ergast()
races = ergast.get_race_schedule(SEASON)  # Races in year 2022
#ergast.get_constructor_info(2022)
results = []

In [35]:

# For each race in the season
for rnd, race in races['raceName'].items():

    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_race_results(season=SEASON, round=rnd + 1)
    if len(temp.content) < 1:
        break
    else:
        temp = temp.content[0]

    # If there is a sprint, get the results as well
    sprint = ergast.get_sprint_results(season=SEASON, round=rnd + 1)
    if sprint.content and sprint.description['round'][0] == rnd + 1:
        temp = pd.merge(temp, sprint.content[0], on='driverCode', how='left')
        # Add sprint points and race points to get the total
        temp['points'] = temp['points_x'] + temp['points_y']
        temp.drop(columns=['points_x', 'points_y'], inplace=True)

    # Add round no. and grand prix name
    temp['round'] = rnd + 1
    temp['race'] = race.removesuffix(' Grand Prix')
    temp = temp[['round', 'race', 'driverCode', 'points']]  # Keep useful cols.
    results.append(temp)

# Append all races into a single dataframe
results = pd.concat(results)
races = results['race'].drop_duplicates()

In [36]:
results = results.pivot(index='driverCode', columns='round', values='points')
# Here we have a 22-by-22 matrix (22 races and 22 drivers, incl. DEV and HUL)


In [37]:


# Rank the drivers by their total points
results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races

race,Bahrain,Saudi Arabian,Australian,Emilia Romagna,Miami,Spanish,Monaco,Azerbaijan,Canadian,British,...,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,United States,Mexico City,São Paulo,Abu Dhabi
driverCode,,,,,,,,,,,,,,,,,,,,,
VER,0.0,25.0,0.0,34.0,26.0,25.0,15.0,25.0,25.0,6.0,...,25.0,26.0,26.0,25.0,6.0,25.0,25.0,25.0,13.0,25.0
LEC,26.0,19.0,26.0,15.0,18.0,0.0,12.0,0.0,10.0,12.0,...,8.0,8.0,15.0,18.0,18.0,15.0,15.0,8.0,15.0,18.0
PER,0.0,12.0,18.0,24.0,12.0,19.0,25.0,19.0,0.0,18.0,...,10.0,18.0,10.0,9.0,25.0,18.0,12.0,15.0,10.0,15.0
RUS,12.0,10.0,15.0,12.0,10.0,15.0,10.0,15.0,12.0,0.0,...,15.0,12.0,18.0,15.0,0.0,4.0,11.0,13.0,34.0,10.0
SAI,18.0,15.0,0.0,5.0,15.0,12.0,18.0,0.0,19.0,25.0,...,12.0,15.0,4.0,12.0,15.0,0.0,0.0,10.0,22.0,12.0
HAM,15.0,1.0,12.0,0.0,8.0,10.0,4.0,12.0,15.0,16.0,...,19.0,0.0,12.0,10.0,2.0,10.0,18.0,18.0,24.0,0.0
NOR,0.0,6.0,10.0,19.0,0.0,4.0,9.0,2.0,0.0,8.0,...,6.0,0.0,6.0,6.0,12.0,1.0,8.0,2.0,2.0,9.0
OCO,6.0,8.0,6.0,0.0,4.0,6.0,0.0,1.0,8.0,0.0,...,2.0,6.0,2.0,0.0,0.0,12.0,0.0,4.0,4.0,6.0
ALO,2.0,0.0,0.0,0.0,0.0,2.0,6.0,6.0,2.0,10.0,...,4.0,10.0,8.0,0.0,0.0,6.0,6.0,0.0,10.0,0.0


In [39]:
import plotly.express as px
import webbrowser  # Import the webbrowser module to open the HTML file in the browser

# Assuming 'results' is your data, replace it with your actual dat
fig = px.imshow(
    results,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Points'}       # Change hover texts
)
fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')
fig.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig.update_layout(coloraxis_showscale=False)        # Remove legend
fig.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig.write_html('plot.html')

# Open the HTML file in the default web browser
webbrowser.open('plot.html')


True

# ALL the plot is working good

In [3]:
def plot_standings_by_driver(SEASON):
    ergast = Ergast()
    races = ergast.get_race_schedule(SEASON)  # Races in year 2022
    results = []
    for rnd, race in races['raceName'].items():

        # Get results. Note that we use the round no. + 1, because the round no.
        # starts from one (1) instead of zero (0)
        temp = ergast.get_race_results(season=SEASON, round=rnd + 1)
        if len(temp.content) < 1:
            break
        else:
            temp = temp.content[0]

        # If there is a sprint, get the results as well
        sprint = ergast.get_sprint_results(season=SEASON, round=rnd + 1)
        if sprint.content and sprint.description['round'][0] == rnd + 1:
            temp = pd.merge(temp, sprint.content[0], on='driverCode', how='left')
            # Add sprint points and race points to get the total
            temp['points'] = temp['points_x'] + temp['points_y']
            temp.drop(columns=['points_x', 'points_y'], inplace=True)

        # Add round no. and grand prix name
        temp['round'] = rnd + 1
        temp['race'] = race.removesuffix(' Grand Prix')
        temp = temp[['round', 'race', 'driverCode', 'points']]  # Keep useful cols.
        results.append(temp)

    # Append all races into a single dataframe
    results = pd.concat(results)
    races = results['race'].drop_duplicates()
    results = results.pivot(index='driverCode', columns='round', values='points')
    # Rank the drivers by their total points
    results['total_points'] = results.sum(axis=1)
    results = results.sort_values(by='total_points', ascending=False)
    results.drop(columns='total_points', inplace=True)

    # Use race name, instead of round no., as column names
    results.columns = races
    fig = go.Figure(data=go.Heatmap(
        z=results.values,
        x=results.columns,
        y=results.index,
        colorscale=[[0, 'rgb(198, 219, 239)'],
                    [0.25, 'rgb(107, 174, 214)'],
                    [0.5, 'rgb(33,  113, 181)'],
                    [0.75, 'rgb(8,   81,  156)'],
                    [1, 'rgb(8,   48,  107)']],
        hovertemplate='Race: %{x}<br>Driver: %{y}<br>Points: %{z}<extra></extra>'
    ))

    # Customize the layout
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='', tickmode='array', tickvals=results.index, ticktext=results.index)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', showline=False, tickson='boundaries')
    fig.update_xaxes(showgrid=False, showline=False)
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', coloraxis_showscale=False, xaxis_side='top',
                      margin=dict(l=0, r=0, b=0, t=0))
    return fig

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='season-dropdown',
        options=[
            {'label': '2022', 'value': 2023},
            {'label': '2022', 'value': 2022},
            {'label': '2022', 'value': 2021},
            {'label': '2022', 'value': 2020},
            {'label': '2022', 'value': 2019},
            # Add more options if needed
        ],
        value=2022  # Default selected value
    ),
    dcc.Graph(id='standings-graph')
])


@app.callback(
    Output('standings-graph', 'figure'),
    [Input('season-dropdown', 'value')]
)
def update_graph(selected_season):
    fig = plot_standings_by_driver(selected_season)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)